In [5]:
import sys
# sys.path.remove('/Users/lilianlee/opt/anaconda3/envs/py39/lib/python3.9/site-packages/dysmalpy-1.8.1a4-py3.9-macosx-10.9-x86_64.egg')
sys.path.insert(0,"/Users/lilianlee/git_softwares/dysmalpy")

from dysmalpy.utils import calc_pixel_distance


In [1]:
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy as np

def plot_ellip_apertures(ellip_apertures):
    # Create a new figure and axis
    fig, ax = plt.subplots()

    # Loop through each aperture in ellip_apertures
    for aperture in ellip_apertures.apertures:
        # Extract relevant information
        center = aperture.aper_center
        semi_major_axis = aperture.pix_perp * aperture.pixscale
        semi_minor_axis = aperture.pix_parallel * aperture.pixscale
        angle_deg = aperture.slit_PA

        # Create an Ellipse patch representing the aperture
        ellipse = Ellipse(xy=center, width=2 * semi_major_axis, height=2 * semi_minor_axis,
                          angle=angle_deg, edgecolor='r', facecolor='none', lw=1)

        # Add the Ellipse to the axis
        ax.add_patch(ellipse)

    # Set axis limits based on your data range
    ax.set_xlim(0, ellip_apertures.nx)
    ax.set_ylim(0, ellip_apertures.ny)

    # Set axis labels or other settings as needed
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')

    # Show the plot
    plt.show()


# ellip_apertures = EllipApertures(...)
# plot_ellip_apertures(ellip_apertures)


In [ ]:

class EllipApertures(Apertures):
    """
    Set of elliptical apertures. 
    
    Uses same generic extract_1d_kinematics as Apertures.
    Sizes can vary. -- depending on if pix_perp and pix_parallel are arrays or scalar.

    FOR THIS CASE: aper_centers are along the slit.

    rarr should be in *** ARCSEC ***

    Note here that pix_perp and pix_parallel are the semi axes lengths!


    """
    def __init__(self, rarr=None, slit_PA=None, pix_perp=None, pix_parallel=None,
             nx=None, ny=None, center_pixel=None, pixscale=None, partial_weight=True, rotate_cube=False,
             moment=False):

        #
        if rotate_cube:
            slit_PA_use = 0.
        else:
            slit_PA_use = slit_PA

        # Assume the default central pixel is the center of the cube
        if center_pixel is None:
            center_pixel = ((nx - 1) / 2., (ny - 1) / 2.)

        try:
            if len(pix_perp) > 1:
                pix_perp = np.array(pix_perp)
            else:
                pix_perp = np.repeat(pix_perp[0], len(rarr))
        except:
            pix_perp = np.repeat(pix_perp, len(rarr))

        try:
            if len(pix_parallel) > 1:
                pix_parallel = np.array(pix_parallel)
            else:
                pix_parallel = np.repeat(pix_parallel[0], len(rarr))
        except:
            pix_parallel = np.repeat(pix_parallel, len(rarr))

        self.pix_perp = pix_perp
        self.pix_parallel = pix_parallel
        self.rarr = rarr
        self.nx = nx
        self.ny = ny
        self.center_pixel = center_pixel
        self.pixscale = pixscale

        aper_centers_pix = np.zeros((2,len(self.rarr)))
        apertures = []
        for i in range(len(rarr)):
            aper_cent_pix = [rarr[i]*np.sin(slit_PA_use*deg2rad)/self.pixscale + self.center_pixel[0],
                            rarr[i]*-1.*np.cos(slit_PA_use*deg2rad)/self.pixscale + self.center_pixel[1]]
            aper_centers_pix[:,i] = aper_cent_pix
            apertures.append(EllipAperture(slit_PA=slit_PA_use,
                        pix_perp=self.pix_perp[i], pix_parallel=self.pix_parallel[i],
                        aper_center=aper_cent_pix, nx=self.nx, ny=self.ny, partial_weight=partial_weight,
                        moment=moment))

        self.aper_centers_pix = aper_centers_pix

        super(EllipApertures, self).__init__(apertures=apertures, slit_PA=slit_PA, rotate_cube=rotate_cube)

In [ ]:
class EllipAperture(Aperture):
    """
    Elliptical Aperture

    pix_perp and pix_parallel are number of pixels of the elliptical semi axes lengths.

    Note: slit_PA is CCW from north / up (sky "y" direction). In Degrees!
    """

    def __init__(self, slit_PA=None, pix_perp=None, pix_parallel=None,
            aper_center=None, nx=None, ny=None, partial_weight=True,
            moment=False):

        # set things here
        self.pix_perp = pix_perp
        self.pix_parallel = pix_parallel
        self.slit_PA = slit_PA


        super(EllipAperture, self).__init__(aper_center=aper_center,
                                            nx=nx, ny=ny, partial_weight=partial_weight,
                                            moment=moment)


    def define_aperture_mask(self):
        seps_sky, pa_sky = calc_pixel_distance(self.nx, self.ny, self.aper_center)

        xskys = seps_sky * -1 * np.sin(pa_sky*deg2rad)
        yskys = seps_sky * np.cos(pa_sky*deg2rad)

        xslits = xskys * np.cos(self.slit_PA*deg2rad)       + yskys * np.sin(self.slit_PA*deg2rad)
        yslits = xskys * -1. * np.sin(self.slit_PA*deg2rad) + yskys * np.cos(self.slit_PA*deg2rad)

        try:
            if self.partial_weight:
                do_partial_weight = True
            else:
                do_partial_weight = False
        except:
            do_partial_weight = True

        if do_partial_weight:
            if shapely_installed:
                apmask = np.ones(xskys.shape) * -99.

                wh_allin = np.where( (np.abs(yslits)/self.pix_parallel + 1./np.sqrt(2.))**2 + \
                                     (np.abs(xslits)/self.pix_perp + 1./np.sqrt(2.))**2 <= 1. )
                wh_allout = np.where( (np.abs(yslits)/self.pix_parallel - 1./np.sqrt(2.))**2 + \
                                      (np.abs(xslits)/self.pix_perp - 1./np.sqrt(2.))**2 > 1. )

                apmask[wh_allin] = 1.
                apmask[wh_allout] = 0.

                wh_calc = np.where(apmask < 0)

                # define shapely object:
                circtmp = Point((0,0)).buffer(1)
                aper_ell = shply_affinity.scale(circtmp, int(self.pix_perp), int(self.pix_parallel))


                for (whc_y, whc_x) in zip(*wh_calc):
                    x_sky_pix_corners = np.array([xskys[whc_y,whc_x]+0.5, xskys[whc_y,whc_x]+0.5,
                                                  xskys[whc_y,whc_x]-0.5, xskys[whc_y,whc_x]-0.5] )
                    y_sky_pix_corners = np.array([yskys[whc_y,whc_x]+0.5, yskys[whc_y,whc_x]-0.5,
                                                  yskys[whc_y,whc_x]-0.5, yskys[whc_y,whc_x]+0.5] )
                    x_slit_pix_corners = x_sky_pix_corners * np.cos(self.slit_PA*deg2rad)       + y_sky_pix_corners * np.sin(self.slit_PA*deg2rad)
                    y_slit_pix_corners = x_sky_pix_corners * -1. * np.sin(self.slit_PA*deg2rad) + y_sky_pix_corners * np.cos(self.slit_PA*deg2rad)

                    cornerspix = np.array([x_slit_pix_corners, y_slit_pix_corners])
                    pix_poly = Polygon(cornerspix.T)

                    overlap = aper_ell.intersection(pix_poly)
                    apmask[whc_y,whc_x] = overlap.area
            else:
                raise ValueError("Currently cannot do partial weights if python package shapely is not installed")
                # apmask = None # fractional pixels
        else:
            apmask = ( (yslits/self.pix_parallel)**2 + (xslits/self.pix_perp)**2 <= 1. )

        return apmask